In [1]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=options)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu180

In [2]:
!pip install --user requests_html
!sudo pip install colorama

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.3.0 requires urllib3[secure,socks]~=1.26, but you have urllib3 1.25.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import re
import time
import requests
import colorama
import traceback
import requests_html
from time import sleep
from lxml import etree
from colorama import Fore
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import ChromeOptions

def preproc(text):
    text = re.sub(r"'|\\u003+[a-z]{1,}|/+[a-z]{1,}|\\+[a-z]", '', text).replace("'", '').replace('"', "").replace(':', ': ').replace('.', '. ').replace("  ", " ")
    return text

def get_doctor_answer():
    soup = BeautifulSoup(driver.page_source, "lxml")
    result = str(soup.find_all('doctor-topic')[0]).split(" ")

    left, right = None, None

    for i, j in enumerate(result):
        if re.match("doctor-topic-content=", j):
            left = i
        if re.match("post-date=", j):
            right = i

    result = " ".join(result[left+1:right])
    return preproc(result)

def get_patient_question():
    soup = BeautifulSoup(driver.page_source, "lxml")
    result = str(soup.find_all('detail-topic')[0]).split(" ")

    left, right = None, None
    for i, j in enumerate(result):
        if re.match("member-topic-content=", j):
            left = i
        if re.match("member-topic-title=", j):
            right = i

    result = " ".join(result[left+1:right])
    return preproc(result)

def get_href(url):
    global is_answered
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    result = soup.find_all("card-topic")
    results = []

    for i in range(len(result)):
        is_answered = res.html.find('#topic-list > card-topic:nth-child({})'.format(i+1), first=True)
        is_answered = res.html.xpath('//*[@id="topic-list"]/card-topic[{}]'.format(i+1), first=True)
        if int(is_answered.attrs['counter-reply']) > 0:
            result = str(soup.find_all("card-topic")[i]).split(" ")
            left, right = None, None
            for i, j in enumerate(result):
                if re.match("href=", j):
                    left = i
                if re.match("image-url=", j):
                    right = i
            result = " ".join(result[left:right])
            result = re.sub('href=|"', "", result)
            result = "https://www.alodokter.com" + result
            results.append(result)
    return results

# option = webdriver.ChromeOptions()
# option.add_argument("headless")

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
options.add_argument(f'--user-agent={user_agent}')

answers = []
questions = []

p = 1
pages = 1000
pbar = tqdm(total=pages)
while p < pages: # 21977 is max pagination so far
    if p == 1:
        url = "https://www.alodokter.com/komunitas/diskusi/penyakit"
    url = "https://www.alodokter.com/komunitas/diskusi/penyakit/page/{}".format(p)
    chrome_options = webdriver.ChromeOptions()  
    # ChromeOptions.binary_location = "C:/Program Files/Google/Chrome Beta/Application/chrome.exe"
    driver = webdriver.Chrome('chromedriver', options=options)
    # driver = webdriver.Chrome("./chromedriver.exe", options=option)
    driver.get(url)

    ses = requests_html.HTMLSession()
    res = ses.get(url)
    html = res.html

    URL = get_href(url)
    for card_url in URL:
        try:
            chrome_options = webdriver.ChromeOptions()
            # ChromeOptions.binary_location = "C:/Program Files/Google/Chrome Beta/Application/chrome.exe"
            # driver = webdriver.Chrome("./chromedriver.exe")
            driver = webdriver.Chrome('chromedriver', options=options)
            driver.get(card_url)
            sleep(1)
            patient_question = get_patient_question()
            doctor_answer = get_doctor_answer()
            questions.append(patient_question)
            answers.append(doctor_answer)
            driver. close()
            print(card_url + Fore.LIGHTGREEN_EX + " ==SUCCEED==")
        except Exception as e:
            print(card_url + Fore.LIGHTRED_EX + " ==FAILED==")
            print(e)
    p+=1
    pbar.update(1)
pbar.close()

  0%|          | 0/1000 [00:00<?, ?it/s]

https://www.alodokter.com/komunitas/topic/demam--amp--infeksi-jamur ==SUCCEED==
https://www.alodokter.com/komunitas/topic/suntik-intravena--iv- ==SUCCEED==
https://www.alodokter.com/komunitas/topic/takut-ambil-keputusan-karena-banyak-pertimbangan ==SUCCEED==
https://www.alodokter.com/komunitas/topic/foto-hasil-thoraks ==SUCCEED==
https://www.alodokter.com/komunitas/topic/kehamilan2389ce ==SUCCEED==
https://www.alodokter.com/komunitas/topic/hamil52522e ==SUCCEED==
https://www.alodokter.com/komunitas/topic/lapar-karena-darah ==SUCCEED==
https://www.alodokter.com/komunitas/topic/minuman-ganda ==SUCCEED==
https://www.alodokter.com/komunitas/topic/mengapa-selalu-merasa-diawasi-saat-di-lingkungan-sosial- ==SUCCEED==
https://www.alodokter.com/komunitas/topic/telinga-sakit-akbiat-serumen-prop ==SUCCEED==
https://www.alodokter.com/komunitas/topic/cek-hsv-2 ==SUCCEED==
https://www.alodokter.com/komunitas/topic/haid-setelah-melahirkan16a72a ==SUCCEED==
https://www.alodokter.com/komunitas/topic/ha